In [10]:
import torch
import torch.nn as nn
from neuralop.models import FNO
import numpy as np
import re
import scipy
from torch.utils.data import Dataset, DataLoader
import numpy
import matplotlib.pyplot as plt
from tqdm import tqdm  # This imports the tqdm class for progress bars
from Operator import utilities3

plt.rc('font', family='STIXGeneral')

In [9]:
import sys
sys.path.append('../')
import sys
import os

# 获取最后添加的路径的绝对路径
absolute_path = os.path.abspath(sys.path[-1])

print("添加的绝对路径是:", absolute_path)

添加的绝对路径是: /liujinxin/lam/OE


In [12]:

class OE_Dataset(Dataset):
    def __init__(self, input_data,input_cond,out,task="NCHD",**kwargs):
      

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.input_data = input_data.to(self.device) #[b, 1,1, 300] 时间维度已经在前面
        self.condition = input_cond
        self.ground_truth = out.to(self.device)# ground truth
        self.task = task
         #这里condition的是A和L 两个条件
        self.con_tensor=self.conditions_to_tensor()
    
       
        
    def __len__(self):
        
        return  self.input_data.shape[0]

    def __getitem__(self, idx):
    
       
        ini_data = self.input_data[idx] # ini_eta 
        ground_truth = self.ground_truth[idx]
        condition = self.con_tensor[idx] # condition,已经进入cuda
   
        
        return ini_data,condition,ground_truth
    
    def conditions_to_tensor(self):
       # 假设每个条件都有2个值（A和L）
        if self.task =="NCHD":# 2个条件
            
            tensor = torch.zeros((len(self.condition), 2), dtype=torch.float32).to(self.device)
       
        
        for i, (key, value) in enumerate(self.condition.items()):
            tensor[i, 0] = torch.tensor(value['A'], dtype=torch.float32)  # 第一个值是A
            tensor[i, 1] = torch.tensor(value['L'], dtype=torch.float32)  # 第二个值是L

        return tensor
#查找该目录里面所有.mat 文件 返回name list
def return_name_list(folder):
    # 初始化一个空列表，用于存储找到的.mat文件名
    name_list = []
    
    # os.walk遍历文件夹
    for dirpath, dirnames, filenames in os.walk(folder):
        # 遍历文件名
        for filename in filenames:
            # 检查文件扩展名是否为.mat
            if filename.endswith('.mat'):
                # 如果是，将文件名添加到列表中
                name_list.append(folder+"/"+filename)
                
    return name_list

   
class Read_Mat_4torch():
    def __init__(self, mat_file:list):
        
        self.mat_file = mat_file
        # 正则表达式模式
        self.CONDITIONS = {}
        self.datas = None  # 初始化datas为None
        for i,mat in enumerate(mat_file):
            pattern = r'A_([0-9.]+)_L_([0-9.]+)'

            # 搜索模式并提取数字
            match = re.search(pattern, mat)
            self.A = np.float32(match.group(1))
            self.L = np.float32(match.group(2))
            self.CONDITIONS[f"{mat}"]={ "A":self.A,
                                        "L":self.L}
        
    def _return_conditions(self):
        return self.CONDITIONS 

    def _read_mat(self):
        
        files = len(self.mat_file)
        self.datas= np.zeros((files,640,300,3))
        
        for i, mat in tqdm(enumerate(self.mat_file), total=len(self.mat_file), desc="Loading MAT files"):
            
            self.data = scipy.io.loadmat(mat)
            self.wave_data = self.data['wave_data'][0, 0]

            title = ["deepsea", "slope", "normal"]

            self.deepsea_data = self.wave_data['deepsea'].reshape(640, 100, 3).astype(np.float32)
            self.slope_data = self.wave_data['slope'].reshape(640, 100, 3).astype(np.float32)
            self.normal_data = self.wave_data['normal'].reshape(640, 100, 3).astype(np.float32)

            current_data = np.concatenate((self.deepsea_data, self.slope_data, self.normal_data), axis=1)
            self.datas[i] = current_data
            
            #tqdm.write(f"Processed {i+1}/{len(self.mat_file)} files")

        self.datas = torch.from_numpy(self.datas).float() #转torch
        self.datas = self.datas.permute(0,3,1,2) # 
        print("data shape",self.datas.shape)
  
        return self.datas,self.CONDITIONS

In [13]:
All_mat_file= return_name_list(folder="/liujinxin/lam/OE/Data/Train/NCHD")
# 打乱文件列表以确保随机性
np.random.shuffle(All_mat_file)

# 计算训练集的大小（70%）
train_size = int(len(All_mat_file) * 0.7)

# 分割列表为训练集和测试集
train_files = All_mat_file[:train_size]
test_files = All_mat_file[train_size:]

In [14]:
train_files

['/liujinxin/lam/OE/Data/Train/NCHD/A_0.030_L_7.76_dr_0.04_Time_scale_170.28023_wave_data.mat',
 '/liujinxin/lam/OE/Data/Train/NCHD/A_0.010_L_3.20_dr_0.04_Time_scale_294.93401_wave_data.mat',
 '/liujinxin/lam/OE/Data/Train/NCHD/A_0.015_L_4.20_dr_0.04_Time_scale_240.81261_wave_data.mat',
 '/liujinxin/lam/OE/Data/Train/NCHD/A_0.020_L_9.20_dr_0.04_Time_scale_208.54984_wave_data.mat',
 '/liujinxin/lam/OE/Data/Train/NCHD/A_0.020_L_1.60_dr_0.04_Time_scale_208.54984_wave_data.mat',
 '/liujinxin/lam/OE/Data/Train/NCHD/A_0.015_L_9.24_dr_0.04_Time_scale_240.81261_wave_data.mat',
 '/liujinxin/lam/OE/Data/Train/NCHD/A_0.015_L_8.12_dr_0.04_Time_scale_240.81261_wave_data.mat',
 '/liujinxin/lam/OE/Data/Train/NCHD/A_0.025_L_6.14_dr_0.04_Time_scale_186.53264_wave_data.mat',
 '/liujinxin/lam/OE/Data/Train/NCHD/A_0.025_L_5.78_dr_0.04_Time_scale_186.53264_wave_data.mat',
 '/liujinxin/lam/OE/Data/Train/NCHD/A_0.025_L_4.14_dr_0.04_Time_scale_186.53264_wave_data.mat',
 '/liujinxin/lam/OE/Data/Train/NCHD/A_0.

In [21]:
from scipy.io import loadmat

# 替换为你的.mat文件的路径
train_torch_data,train_Conditions = Read_Mat_4torch(mat_file = test_files)._read_mat()

#***** 维度变化，输入【b,1,1,300】-> 输出[b,640,1,300]
#train
train_g_t = train_torch_data[:,2:3,:,:]# ground_truth torch.Size([b, 1,640, 300])
train_g_t = train_g_t.permute(0,2,1,3) #=[b,640,1,300]s

train_input_data = train_torch_data[:,2:3,0:1,:] # ini([b, 1,1, 300]) 0:1 iss time
train_input_data = train_input_data.permute(0,2,1,3) # =[20,1,1,300】时间维度在前面

Loading MAT files: 100%|██████████| 677/677 [00:11<00:00, 61.44it/s]


data shape torch.Size([677, 3, 640, 300])


In [22]:
gt_normalizer = utilities3.UnitGaussianNormalizer(train_g_t)

input_normalizer = utilities3.UnitGaussianNormalizer(train_input_data)

In [23]:
# 替换为你的.mat文件的路径
mat_file_path = 'A_0.010_L_1.00_dr_0.04_Time_scale_294.93401_wave_data.mat'
train_torch_data,train_Conditions = Read_Mat_4torch(mat_file = [mat_file_path])._read_mat()

#***** 维度变化，输入【b,1,1,300】-> 输出[b,640,1,300]
#train
train_g_t = train_torch_data[:,2:3,:,:]# ground_truth torch.Size([b, 1,640, 300])
train_g_t = train_g_t.permute(0,2,1,3) #=[b,640,1,300]s

train_input_data = train_torch_data[:,2:3,0:1,:] # ini([b, 1,1, 300]) 0:1 iss time
train_input_data = train_input_data.permute(0,2,1,3) # =[20,1,1,300】时间维度在前面


train_g_t = gt_normalizer.encode(train_g_t)
train_input_data = input_normalizer.encode(train_input_data)

Loading MAT files: 100%|██████████| 1/1 [00:00<00:00, 55.85it/s]


data shape torch.Size([1, 3, 640, 300])


In [24]:
train_OE_Dataset = OE_Dataset(input_data = train_input_data,
                              input_cond = train_Conditions,
                              out = train_g_t)

In [25]:
class Branch_net(nn.Module):
    def __init__(self,input,hidden,output):
        super(Branch_net,self).__init__()
            
        self._net = nn.Sequential(
                     nn.Linear(input, hidden),
                     nn.Tanh(),
                     nn.Linear(hidden, hidden),
                     nn.Tanh(),
                     nn.Linear(hidden, output),
                     nn.Tanh(),
                    )
    def forward(self,x):
        out=self._net(x)
        return out

class MFNO(nn.Module):
  
  def __init__(self,infeature=2,Trunk="FNO",t_steps=640,wavelet="db6",fno_modes=64):
      super(MFNO,self).__init__()
      self.bran_nn = Branch_net(infeature,50,1)
      self.Trunk = Trunk
      if self.Trunk  == "FNO":
        #modes64 的参数量 2605052
        print("FNO")
        self.trunk_nn = FNO(n_modes=(fno_modes,fno_modes),hidden_channels=12,in_channels=1,out_channels=t_steps)
        
      elif self.Trunk  == "UNO":
        print("UNO")
        #modes64 的参数量 2656746
        self.trunk_nn = UNO(1,640, hidden_channels=10, projection_channels=12,uno_out_channels = [8,16,16,16,8], \
            uno_n_modes= [[fno_modes,fno_modes],[32,32],[32,32],[32,32],[fno_modes,fno_modes]], uno_scalings=  [[1.0,1.0],[0.5,0.5],[1,1],[2,2],[1,1]],\
            horizontal_skips_map = None, n_layers = 5, domain_padding = 1)
        
      elif self.Trunk  == "WNO":
        h = 2 # dwt需要偶数 
        s = 300
        print("wno")
      
        layers = 6 #level = 2 , 2606035 参数量 fno modes为分解的level：1/2/3/4 out is tsteps
        self.trunk_nn = WNO2d(width= 6, level = fno_modes, layers = layers , size=[h,s], wavelet= wavelet,
              in_channel = 3, grid_range=[1,1], padding=0,out=640).to(device) #实际输出【1，1，3，640】
        #输入[1, 1,1, 300]输出[1,640,1,300]
        
      
      self.condition_norm = nn.BatchNorm1d(infeature)  # 归一化层
      self.ini_norm = nn.BatchNorm2d(1)

  def forward(self,x,condition):
      #对序列进行fno
      # input:[batch,1,1,300], irst 1 is time
      x_norm =self.ini_norm(x)
      condition_normal= self.condition_norm(condition)
      #norm
      if (self.Trunk  == "WNO"):
        #wno输入应该是【b，1，300，1】#time 放最后
        x_norm = x_norm.permute(0,2,3,1)
        trunk_out = self.trunk_nn(x_norm)
        #note： wno 去掉一个维度,因为输入得是偶数
        trunk_out = trunk_out[:,0:1,:,:] #[b,1,300.640]
        trunk_out =trunk_out.permute(0,3,1,2)#out [b,640,3,300]
      else:
        
         trunk_out = self.trunk_nn(x_norm) #[b, 640, 3, 300]，640是时间步
        
      
      out = self.bran_nn(condition_normal)
  
      out = out.unsqueeze(-1)
      out = out.unsqueeze(-1)
      #([batch, 640, 1, 300])
      final_out = trunk_out*out
   
      return final_out

In [64]:
config={'seed': 1,
      'gamma': 100,
      'lambda': 10,
      'bound': [[0,0.02],[1,10]],
      'num_particles': 1000,
      'dimensions': 2,
      'iterations':50}

In [69]:
torch.cuda.reset_max_memory_allocated()  # 重置最大内存计数器

torch.cuda.reset_accumulated_memory_stats()  # 重置累积内存统计

In [70]:
train_g_t.shape

torch.Size([1, 640, 1, 300])

In [71]:
import IMNO

imno=IMNO.Expr_Inverse(config=config,
                  save_name="",
                  net="/liujinxin/lam/OE/Inverse/mno_ckpt.pth",
                  ini=train_input_data.cuda(),
                  eta=train_g_t.cuda(),
                  true_x=torch.tensor([0.01,1]).cuda())

FNO
num_particle: 1000


OutOfMemoryError: CUDA out of memory. Tried to allocate 734.00 MiB (GPU 0; 31.74 GiB total capacity; 30.32 GiB already allocated; 295.38 MiB free; 30.42 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [67]:
imno.search()

___x
___x
num_particle: 1000
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_75540/2461830387.py", line 1, in <module>
    imno.search()
  File "/liujinxin/lam/OE/Inverse/IMNO.py", line 201, in search
  File "/liujinxin/lam/OE/Inverse/IMNO.py", line 222, in optimize
  File "/liujinxin/lam/OE/Inverse/IMNO.py", line 182, in PIMNO_function
    self.hat_eta = self.branch_net(x).unsqueeze(-1) .unsqueeze(-1) * self.ini_truc
torch.cuda.OutOfMemoryError: CUDA out of memory. Tried to allocate 734.00 MiB (GPU 0; 31.74 GiB total capacity; 30.29 GiB already allocated; 301.38 MiB free; 30.41 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

During handling of the above exception, another exception occurred:

Trace